# Tutorial on manipulating SLSTR L2 FRP product from the cloud

In [ ]:
import xarray as xr
from pathlib import Path
from xarray_eop.eop import open_eop_datatree

### Get AWS credentials

In [ ]:
import s3fs
import json
# Assuming credentials are in ~/.eopf/secrets.json
try:
    SECRET_PATH = Path.home() / Path(".eopf/secrets.json")
    with open(SECRET_PATH) as f:
        secrets=json.load(f)

# Or use ENV variables
except:
    import os
    secrets = {"s3input" : {
        "key": os.getenv("AWS_ACCESS_KEY_ID"),
        "secret": os.getenv("AWS_SECRET_ACCES_KEY"),
        "endpoint_url": os.getenv("AWS_ENDPOINT_URL"),
        "region_name": os.getenv("AWS_DEFAULT_REGION")
        }
    }

secrets["s3input"].pop("region_name",None)


### Browse S3 buckets and get SLSFRP product

In [ ]:
import s3fs
SAMPLE_PATH = "s3://s3-input/Products/NewFormat/"
s3 = s3fs.S3FileSystem(
    key=secrets["s3input"]["key"],
    secret=secrets["s3input"]["secret"],
    endpoint_url=secrets["s3input"]["endpoint_url"]
)
s3_path = s3.glob(SAMPLE_PATH+"S03SLSFRP*.zip")
store=f"zip::s3://{s3_path[0]}"
store

### Opening the whole product with datatree

In [ ]:
# Custom open datatree
dt = open_eop_datatree(store,backend_kwargs={"storage_options": {"s3":secrets["s3input"]}})
# Standard open datatree
# dt = datatree.open_datatree(store,engine="zarr",chunks={},backend_kwargs={"storage_options": {"s3":secrets["s3input"]}})
dt

### Opening measurement data (1D) with xarray

In [ ]:
ds=dt.measurements.inadir

### Plot Active Fire pixels positions on a PlateCarree grid

In [ ]:
import matplotlib.pyplot as plt
import cartopy.crs as ccrs

ds.frp_mwir
fig = plt.figure()
ax = plt.axes(projection=ccrs.PlateCarree())
# ax.set_global()
ax.coastlines()
ax.gridlines(draw_labels=True)
# ds.frp_mwir.plot.scatter(x="longitude",y="latitude",transform=ccrs.PlateCarree())
plt.scatter(ds.longitude,ds.latitude,c=ds.frp_mwir,vmax=100)
plt.colorbar()
